In [7]:


def save_data(data, file_path):
    """Save data to a JSON file."""
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
        
def read_data(file_path):
    """Read data from a JSON file."""
    with open(file_path, 'r') as f:
        return json.load(f)

# streams = read_data('./1_flows_data.json')
# filtered_streams = read_data('./2_filtered_flows_data.json')
# occurrences = read_data('./3-occurrences_data.json')


In [8]:
from scapy.all import sniff, IP, TCP
import json
import time

# Constants
WAITING_TIME = 30  # Time threshold to consider packets part of the same flow


def create_flow_key_from_packet(packet):
    """Generate a unique flow key based on packet IP and TCP headers."""
    return f"{packet[IP].src}:{packet[TCP].sport}-{packet[IP].dst}:{packet[TCP].dport}"

def extract_packet_info(packet):
    """Extract necessary information from the packet."""
    return {
        'src_ip': packet[IP].src,
        'src_port': packet[TCP].sport,
        'dst_ip': packet[IP].dst,
        'dst_port': packet[TCP].dport,
        'protocol': packet[IP].proto,
        'time': float(packet.time),  # Convert to float
        'size': len(packet)
    }

def update_or_create_flow(flows, flow_key, packet_info):
    """Update an existing flow or create a new one based on the WAITING_TIME."""
    current_time = packet_info['time']
    if flow_key in flows:
        last_flow = flows[flow_key][-1]
        if current_time - last_flow['end_time'] < WAITING_TIME:
            last_flow['packets'].append(packet_info)
            last_flow['end_time'] = current_time
            last_flow['total_bytes'] += packet_info['size']
        else:
            new_flow = {
                'packets': [packet_info],
                'start_time': current_time,
                'end_time': current_time,
                'total_bytes': packet_info['size'],
                'src_ip': packet_info['src_ip'],
                'dst_ip': packet_info['dst_ip'],
                'protocol': packet_info['protocol']
            }
            flows[flow_key].append(new_flow)
    else:
        flows[flow_key] = [{
            'packets': [packet_info],
            'start_time': current_time,
            'end_time': current_time,
            'total_bytes': packet_info['size'],
            'src_ip': packet_info['src_ip'],
            'dst_ip': packet_info['dst_ip'],
            'protocol': packet_info['protocol']
        }]
    return flows

def manage_flow(packet, flows):
    """Process each packet to manage flow data."""
    if IP in packet and TCP in packet:
        flow_key = create_flow_key_from_packet(packet)
        packet_info = extract_packet_info(packet)
        flows = update_or_create_flow(flows, flow_key, packet_info)
    return flows

def read_pcap_file(pcap_file):
    """Read packets from a pcap file and manage flows."""
    flows = {}
    def packet_processor(packet):
        nonlocal flows
        flows = manage_flow(packet, flows)
    sniff(offline=pcap_file, prn=packet_processor, store=False)
    return flows

# Example usage
pcap_file = 'EX-3.pcap'
flows = read_pcap_file(pcap_file)
save_data(flows, './1_flows_data.json')


In [10]:
# Constants for filtering
MAX_BYTES_THRESHOLD = 5000  # Maximum bytes to consider a flow as relevant
MAX_DURATION_THRESHOLD = 300  # Maximum duration in seconds to consider a flow as relevant
MIN_PACKET_COUNT_THRESHOLD = 5  # Minimum packet count to consider a flow as relevant

def filter_flows(flows):
    """Filter flows based on size, duration, and packet count."""
    filtered_flows = {}
    for flow_key, flow_list in flows.items():
        filtered_flow_list = []
        for flow in flow_list:
            total_bytes = flow['total_bytes']
            duration = flow['end_time'] - flow['start_time']
            packet_count = len(flow['packets'])

            # Filter based on the defined thresholds
            if (total_bytes <= MAX_BYTES_THRESHOLD and 
                duration <= MAX_DURATION_THRESHOLD and 
                packet_count >= MIN_PACKET_COUNT_THRESHOLD):
                filtered_flow_list.append(flow)
        
        if filtered_flow_list:
            filtered_flows[flow_key] = filtered_flow_list
    
    return filtered_flows

# Example usage
filtered_flows = filter_flows(flows)
save_data(filtered_flows, './2_filtered_flows_data.json')

In [19]:
from collections import defaultdict

import math

T_dep = 120  # حداکثر فاصله زمانی مجاز بین دو جریان برای احتمال وابستگی (ثانیه)
N_dep = 10   # حداکثر تفاوت در تعداد تکرار بین دو جریان
Sdep_th = 0.5  # حداقل امتیاز وابستگی برای تشخیص به عنوان وابستگی واقعی

def compute_occurrences(flows):
    """Compute the number of occurrences for each flow."""
    occurrences = {}
    for flow_key, sessions in flows.items():
        occurrences[flow_key] = sum(len(session['packets']) for session in sessions)
    return occurrences

flow_occurrences = compute_occurrences(filtered_flows)
save_data(flow_occurrences, './3_flow_occurrences_data.json')


In [23]:
import math

def extract_two_level_dependencies(flows, T_dep, N_dep, Sdep_th):
    """Extract two-level flow dependencies based on the specified thresholds."""
    occurrences = compute_occurrences(flows)
    dependencies = {}

    # Sort all flows by start time within each host
    for host, sessions in flows.items():
        sorted_flows = sorted(sessions, key=lambda x: x['start_time'])

        # Compare each pair of flows within the sorted list
        for i in range(len(sorted_flows)):
            for j in range(i + 1, len(sorted_flows)):
                fi = sorted_flows[i]
                fj = sorted_flows[j]

                # Check the time gap condition
                if (fj['start_time'] - fi['end_time']) <= T_dep:
                    flow_key_i = fi['flow_key']
                    flow_key_j = fj['flow_key']
                    Ni = occurrences[flow_key_i]
                    Nj = occurrences[flow_key_j]

                    # Check the occurrence difference condition
                    if abs(Ni - Nj) <= N_dep:
                        dependency_key = (flow_key_i, flow_key_j)
                        if dependency_key in dependencies:
                            dependencies[dependency_key] += 1
                        else:
                            dependencies[dependency_key] = 1

    # Calculate the dependency score and filter by threshold
    final_dependencies = {}
    for (fi, fj), Tij in dependencies.items():
        Ni = occurrences[fi]
        Nj = occurrences[fj]
        Sdep = math.sqrt(Tij ** 2 / (Ni * Nj))
        if Sdep > Sdep_th:
            final_dependencies[(fi, fj)] = Sdep

    return final_dependencies

# Example usage
dependencies = extract_two_level_dependencies(flows, T_dep, N_dep, Sdep_th)

# print("Extracted Dependencies:")
# for dep, score in dependencies.items():
#     print(f"Dependency between {dep[0]} and {dep[1]} with score {score}")


KeyError: 'flow_key'